In [2]:
import pandas as pd
import numpy as np
import jieba 
import re 
import json
from sklearn.feature_extraction.text import TfidfVectorizer

In [7]:
def handle(contents):
    new_contents = []
    for content in contents:
        content = ''.join(re.findall(r'[\d|\w]+',content))
        content = ' '.join(jieba.cut(content))
        new_contents.append(content)
    return new_contents

In [8]:
def kind(data):
    global type_dict
    global type_count
    typing = json.loads(data['feature'])['type']
    if  typing not in type_dict:
        type_dict[typing] = type_count
        type_count +=1
    return type_dict[typing]

In [9]:
type_dict = dict()
type_count = 0
fname = r'D:\dataset\sqlResult_1558435.csv'
database = pd.read_csv(fname,encoding = 'gb18030',usecols = ['feature','content'])
database = database.fillna('')
database['target']= database.apply(kind,axis = 1)
contents = database['content'].tolist()
new_contents = handle(contents)

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\lenovo\AppData\Local\Temp\jieba.cache
Loading model cost 1.155 seconds.
Prefix dict has been built succesfully.


In [12]:
tv = TfidfVectorizer()
tv_fit = tv.fit_transform(new_contents)

In [16]:
database.head(5)

,content,feature,target
0,此外，自本周（6月12日）起，除小米手机6等15款机型外，其余机型已暂停更新发布（含开发版/...,"{""type"":""科技"",""site"":""cnbeta"",""commentNum"":""37""...","{'科技': 0, '国际新闻': 1, '新闻': 2, '市场': 3, '观点滚动':..."
1,骁龙835作为唯一通过Windows 10桌面平台认证的ARM处理器，高通强调，不会因为只考...,"{""type"":""科技"",""site"":""cnbeta"",""commentNum"":""15""...","{'科技': 0, '国际新闻': 1, '新闻': 2, '市场': 3, '观点滚动':..."
2,此前的一加3T搭载的是3400mAh电池，DashCharge快充规格为5V/4A。\r\n...,"{""type"":""科技"",""site"":""cnbeta"",""commentNum"":""18""...","{'科技': 0, '国际新闻': 1, '新闻': 2, '市场': 3, '观点滚动':..."
3,这是6月18日在葡萄牙中部大佩德罗冈地区拍摄的被森林大火烧毁的汽车。新华社记者张立云摄\r\n,"{""type"":""国际新闻"",""site"":""环球"",""commentNum"":""0"",""j...","{'科技': 0, '国际新闻': 1, '新闻': 2, '市场': 3, '观点滚动':..."
4,（原标题：44岁女子跑深圳约会网友被拒，暴雨中裸身奔走……）\r\n@深圳交警微博称：昨日清...,"{""type"":""新闻"",""site"":""网易热门"",""commentNum"":""978"",...","{'科技': 0, '国际新闻': 1, '新闻': 2, '市场': 3, '观点滚动':..."


In [18]:
from sklearn.cluster import KMeans
from time import time

In [20]:
t0 = time()
km = KMeans(n_clusters = 5)
km.fit(tv_fit)
print('用时{}'.format(time()-t0))

用时4411.813385725021


In [22]:
km.cluster_centers_

array([[5.96563979e-05, 2.62327682e-06, 5.77321990e-07, ...,
        6.38743348e-07, 1.11353917e-06, 1.17632442e-05],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 9.44833295e-07],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00]])

In [26]:
order_centroids = km.cluster_centers_.argsort()[:,::-1]
terms = tv.get_feature_names()
for i in range(5):
    print('cluster %d：'%i,end = ' ')
    for ind in order_centroids[i,:10]:
        print('%s'% terms[ind],end= ' ')
    print()

cluster 0： 新华社 中国 记者 国际 2017 nn 北京 发展 照片 一带 
cluster 1： nba 比赛 球员 篮球 赛季 季后赛 主场 体育 新华社 20162017 
cluster 2： 外代 二线 新华社 2017 照片 法新 戛纳 n5 欧新 n4 
cluster 3： 选手 网球 晋级 比赛 公开赛 战胜 2017 乒乓球 新华社 决赛 
cluster 4： 足球 联赛 球员 外代 20162017 赛季 新华社 当日 路透 主场 
